In [77]:
import utils
import plots
import pandas as pd
import numpy as np

%matplotlib inline

training = utils.load_training_set()

In [102]:
training['UnknownNumberOfDependents'] = (training.NumberOfDependents.isnull() * 1)
training['UnknownMonthlyIncome'] = (training.MonthlyIncome.isnull() * 1)

training['NoDependents'] = training.NumberOfDependents[training.NumberOfDependents == 0]
training.loc[training.NumberOfDependents.isnull(), 'NoDependents'] = 0

training.loc[training.UnknownNumberOfDependents == 1, 'NumberOfDependents'] = 0

training.NoIncome = training.MonthlyIncome
training.loc[training.MonthlyIncome != 0, 'NoIncome'] = 0
training.loc[training.MonthlyIncome == 0, 'NoIncome'] = 1
training.NoIncome.fillna(0)

print('Incomes in No Income %s' % training.NoIncome[training.NoIncome == 1].count())

training.loc[training.UnknownMonthlyIncome == 1, 'MonthlyIncome'] = 0

Incomes in No Income 605


In [107]:
training['ZeroDebtRatio'] = training.DebtRatio
training.loc[training.DebtRatio == 0, 'ZeroDebtRatio'] = 1
training.loc[training.DebtRatio != 0, 'ZeroDebtRatio'] = 0

print('Zero Debt Ratios %s' % training.ZeroDebtRatio[training.ZeroDebtRatio == 1].count())

training['UnknownIncomeDebtRatio'] = training.DebtRatio
training.loc[training.UnknownMonthlyIncome == 0, 'UnknownIncomeDebtRatio'] = 0
training.loc[training.UnknownMonthlyIncome == 1, 'DebtRatio'] = 0

print('UnkMonthlyIncDebtRatio %s' \
      % training.UnknownIncomeDebtRatio[training.UnknownIncomeDebtRatio == 0].count())

Zero Debt Ratios 32222
UnkMonthlyIncDebtRatio 150000


In [110]:
training['WeirdRevolvingUtilization'] = training.RevolvingUtilizationOfUnsecuredLines
training.loc[np.log(training.RevolvingUtilizationOfUnsecuredLines) > 3, 'WeirdRevolvingUtilization'] = 0
  
training.loc[training.RevolvingUtilizationOfUnsecuredLines == 0, 'ZeroRevolvingUtilization'] = 0
training.loc[np.log(training.RevolvingUtilizationOfUnsecuredLines) > 3, \
             'RevolvingUtilizationOfUnsecuredLines'] = 0

training['LogDebt'] = np.log(np.maximum(training.MonthlyIncome, np.ones(len(training.MonthlyIncome))) \
                             * training.DebtRatio)
training.loc[np.isfinite(training.LogDebt), 'LogDebt'] = 0

/media/rodsnjr/Files/miniconda3/envs/udacity-ml/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app
/media/rodsnjr/Files/miniconda3/envs/udacity-ml/lib/python2.7/site-packages/ipykernel/__main__.py:5: RuntimeWarning: divide by zero encountered in log
/media/rodsnjr/Files/miniconda3/envs/udacity-ml/lib/python2.7/site-packages/ipykernel/__main__.py:7: RuntimeWarning: divide by zero encountered in log


In [111]:
training['RevolvingLines'] = training.NumberOfOpenCreditLinesAndLoans - training.NumberRealEstateLoansOrLines

training['HasRevolvingLines'] = (training.RevolvingLines[training.RevolvingLines > 0] * 1)
training['HasRealEstateLoans'] = (training.NumberRealEstateLoansOrLines[training\
                                                                        .NumberRealEstateLoansOrLines > 0] * 1)

In [115]:
training['EligibleSS'] = training.age[training.age > 60].astype(int)
training['DTIOver33'] = training.NoIncome[(training.NoIncome == 0) & (training.DebtRatio > 0.33)].astype(int)
training['DTIOver43'] = training.NoIncome[(training.NoIncome == 0) & (training.DebtRatio > 0.43)].astype(int)
training['DisposableIncome'] = (1 - training.DebtRatio) * training.MonthlyIncome
training.loc[training.NoIncome == 1, 'DisposableIncome'] = 0

training['RevolvingToRealEstate'] = training.RevolvingLines / (1 + training.NumberRealEstateLoansOrLines)

In [117]:
""" R

x$NumberOfTime30.59DaysPastDueNotWorseLarge <- as.integer(x$NumberOfTime30.59DaysPastDueNotWorse > 90)
x$NumberOfTime30.59DaysPastDueNotWorse96 <- as.integer(x$NumberOfTime30.59DaysPastDueNotWorse == 96)
x$NumberOfTime30.59DaysPastDueNotWorse98 <- as.integer(x$NumberOfTime30.59DaysPastDueNotWorse == 98)
x$Never30.59DaysPastDueNotWorse <- as.integer(x$NumberOfTime30.59DaysPastDueNotWorse == 0)
x$NumberOfTime30.59DaysPastDueNotWorse[x$NumberOfTime30.59DaysPastDueNotWorse > 90] <- 0

"""

print 'TODO'

TODO


In [120]:
""" R
  x$NumberOfTime60.89DaysPastDueNotWorseLarge <- as.integer(x$NumberOfTime60.89DaysPastDueNotWorse > 90)
  x$NumberOfTime60.89DaysPastDueNotWorse96 <- as.integer(x$NumberOfTime60.89DaysPastDueNotWorse == 96)
  x$NumberOfTime60.89DaysPastDueNotWorse98 <- as.integer(x$NumberOfTime60.89DaysPastDueNotWorse == 98)
  x$Never60.89DaysPastDueNotWorse <- as.integer(x$NumberOfTime60.89DaysPastDueNotWorse == 0)
  x$NumberOfTime60.89DaysPastDueNotWorse[x$NumberOfTime60.89DaysPastDueNotWorse > 90] <- 0
"""

print 'TODO'

TODO


In [121]:
""" R
  x$NumberOfTimes90DaysLateLarge <- as.integer(x$NumberOfTimes90DaysLate > 90)
  x$NumberOfTimes90DaysLate96 <- as.integer(x$NumberOfTimes90DaysLate == 96)
  x$NumberOfTimes90DaysLate98 <- as.integer(x$NumberOfTimes90DaysLate == 98)
  x$Never90DaysLate <- as.integer(x$NumberOfTimes90DaysLate == 0)
  x$NumberOfTimes90DaysLate[x$NumberOfTimes90DaysLate > 90] <- 0
"""

print 'TODO'

TODO


In [122]:
""" R

  x$IncomeDivBy10 <- as.integer(x$MonthlyIncome %% 10 == 0)
  x$IncomeDivBy100 <- as.integer(x$MonthlyIncome %% 100 == 0)
  x$IncomeDivBy1000 <- as.integer(x$MonthlyIncome %% 1000 == 0)
  x$IncomeDivBy5000 <- as.integer(x$MonthlyIncome %% 5000 == 0)

"""

print 'TODO'

TODO


In [123]:
""" R

  x$Weird0999Utilization <- as.integer(x$RevolvingUtilizationOfUnsecuredLines == 0.9999999)
  x$FullUtilization <- as.integer(x$RevolvingUtilizationOfUnsecuredLines == 1)
  x$ExcessUtilization <- as.integer(x$RevolvingUtilizationOfUnsecuredLines > 1)


"""

print 'TODO'

TODO


In [124]:
""" R


  x$NumberOfTime30.89DaysPastDueNotWorse <- x$NumberOfTime30.59DaysPastDueNotWorse + x$NumberOfTime60.89DaysPastDueNotWorse
  x$Never30.89DaysPastDueNotWorse <- x$Never60.89DaysPastDueNotWorse * x$Never30.59DaysPastDueNotWorse

"""

print 'TODO'

TODO


In [125]:
""" R

  x$NumberOfTimesPastDue <- x$NumberOfTime30.59DaysPastDueNotWorse + x$NumberOfTime60.89DaysPastDueNotWorse + x$NumberOfTimes90DaysLate
  x$NeverPastDue <- x$Never90DaysLate * x$Never60.89DaysPastDueNotWorse * x$Never30.59DaysPastDueNotWorse
  x$Log.RevolvingUtilizationTimesLines <- log1p(x$RevolvingLines * x$RevolvingUtilizationOfUnsecuredLines)

"""

print 'TODO'

TODO


In [126]:
""" R

  x$Log.RevolvingUtilizationOfUnsecuredLines <- log(x$RevolvingUtilizationOfUnsecuredLines)
  x$Log.RevolvingUtilizationOfUnsecuredLines[is.na(x$Log.RevolvingUtilizationOfUnsecuredLines)] <- 0
  x$Log.RevolvingUtilizationOfUnsecuredLines[!is.finite(x$Log.RevolvingUtilizationOfUnsecuredLines)] <- 0
  x$RevolvingUtilizationOfUnsecuredLines <- NULL


"""

print 'TODO'

TODO


In [127]:
""" R

  x$DelinquenciesPerLine <- x$NumberOfTimesPastDue / x$NumberOfOpenCreditLinesAndLoans
  x$DelinquenciesPerLine[x$NumberOfOpenCreditLinesAndLoans == 0] <- 0
  x$MajorDelinquenciesPerLine <- x$NumberOfTimes90DaysLate / x$NumberOfOpenCreditLinesAndLoans
  x$MajorDelinquenciesPerLine[x$NumberOfOpenCreditLinesAndLoans == 0] <- 0
  x$MinorDelinquenciesPerLine <- x$NumberOfTime30.89DaysPastDueNotWorse / x$NumberOfOpenCreditLinesAndLoans
  x$MinorDelinquenciesPerLine[x$NumberOfOpenCreditLinesAndLoans == 0] <- 0

"""

print 'TODO'

TODO


In [128]:
""" R

  x$DelinquenciesPerRevolvingLine <- x$NumberOfTimesPastDue / x$RevolvingLines
  x$DelinquenciesPerRevolvingLine[x$RevolvingLines == 0] <- 0
  x$MajorDelinquenciesPerRevolvingLine <- x$NumberOfTimes90DaysLate / x$RevolvingLines
  x$MajorDelinquenciesPerRevolvingLine[x$RevolvingLines == 0] <- 0
  x$MinorDelinquenciesPerRevolvingLine <- x$NumberOfTime30.89DaysPastDueNotWorse / x$RevolvingLines
  x$MinorDelinquenciesPerRevolvingLine[x$RevolvingLines == 0] <- 0

"""

print 'TODO'

TODO


In [129]:
""" R

  x$Log.DebtPerLine <- x$Log.Debt - log1p(x$NumberOfOpenCreditLinesAndLoans)
  x$Log.DebtPerRealEstateLine <- x$Log.Debt - log1p(x$NumberRealEstateLoansOrLines)
  x$Log.DebtPerPerson <- x$Log.Debt - log1p(x$NumberOfDependents)
  x$RevolvingLinesPerPerson <- x$RevolvingLines / (1 + x$NumberOfDependents)
  x$RealEstateLoansPerPerson <- x$NumberRealEstateLoansOrLines / (1 + x$NumberOfDependents)
  x$UnknownNumberOfDependents <- as.integer(x$UnknownNumberOfDependents)
  x$YearsOfAgePerDependent <- x$age / (1 + x$NumberOfDependents)


"""

print 'TODO'

TODO


In [130]:
""" R

  x$Log.MonthlyIncome <- log(x$MonthlyIncome)
  x$Log.MonthlyIncome[!is.finite(x$Log.MonthlyIncome)|is.na(x$Log.MonthlyIncome)] <- 0
  x$MonthlyIncome <- NULL
  x$Log.IncomePerPerson <- x$Log.MonthlyIncome - log1p(x$NumberOfDependents)
  x$Log.IncomeAge <- x$Log.MonthlyIncome - log1p(x$age)

"""

print 'TODO'

TODO


In [131]:
"""

  x$Log.NumberOfTimesPastDue <- log(x$NumberOfTimesPastDue)
  x$Log.NumberOfTimesPastDue[!is.finite(x$Log.NumberOfTimesPastDue)] <- 0

"""

print 'TODO'

TODO


In [132]:
"""
  x$Log.NumberOfTimes90DaysLate <- log(x$NumberOfTimes90DaysLate)
  x$Log.NumberOfTimes90DaysLate[!is.finite(x$Log.NumberOfTimes90DaysLate)] <- 0

"""

print 'TODO'

TODO


In [133]:
"""

  x$Log.NumberOfTime30.59DaysPastDueNotWorse <- log(x$NumberOfTime30.59DaysPastDueNotWorse)
  x$Log.NumberOfTime30.59DaysPastDueNotWorse[!is.finite(x$Log.NumberOfTime30.59DaysPastDueNotWorse)] <- 0

"""

print 'TODO'

TODO


In [134]:
"""

  x$Log.NumberOfTime60.89DaysPastDueNotWorse <- log(x$NumberOfTime60.89DaysPastDueNotWorse)
  x$Log.NumberOfTime60.89DaysPastDueNotWorse[!is.finite(x$Log.NumberOfTime60.89DaysPastDueNotWorse)] <- 0

"""

print 'TODO'

TODO


In [135]:
"""

  x$Log.Ratio90to30.59DaysLate <- x$Log.NumberOfTimes90DaysLate - x$Log.NumberOfTime30.59DaysPastDueNotWorse
  x$Log.Ratio90to60.89DaysLate <- x$Log.NumberOfTimes90DaysLate - x$Log.NumberOfTime60.89DaysPastDueNotWorse

"""

print 'TODO'

TODO


In [136]:
"""

  x$AnyOpenCreditLinesOrLoans <- as.integer(x$NumberOfOpenCreditLinesAndLoans > 0)
  x$Log.NumberOfOpenCreditLinesAndLoans <- log(x$NumberOfOpenCreditLinesAndLoans)
  x$Log.NumberOfOpenCreditLinesAndLoans[!is.finite(x$Log.NumberOfOpenCreditLinesAndLoans)] <- 0
  x$Log.NumberOfOpenCreditLinesAndLoansPerPerson <- x$Log.NumberOfOpenCreditLinesAndLoans - log1p(x$NumberOfDependents)


"""

print 'TODO'

TODO


In [137]:
"""

  x$Has.Dependents <- as.integer(x$NumberOfDependents > 0)
  x$Log.HouseholdSize <- log1p(x$NumberOfDependents)
  x$NumberOfDependents <- NULL

"""

print 'TODO'

TODO


In [138]:
"""

  x$Log.DebtRatio <- log(x$DebtRatio)
  x$Log.DebtRatio[!is.finite(x$Log.DebtRatio)] <- 0
  x$DebtRatio <- NULL

"""

print 'TODO'

TODO


In [139]:
"""
  x$Log.DebtPerDelinquency <- x$Log.Debt - log1p(x$NumberOfTimesPastDue)
  x$Log.DebtPer90DaysLate <- x$Log.Debt - log1p(x$NumberOfTimes90DaysLate)
"""

print 'TODO'

TODO


In [140]:
"""
  x$Log.UnknownIncomeDebtRatio <- log(x$UnknownIncomeDebtRatio)
  x$Log.UnknownIncomeDebtRatio[!is.finite(x$Log.UnknownIncomeDebtRatio)] <- 0
  x$IntegralDebtRatio <- NULL
  x$Log.UnknownIncomeDebtRatioPerPerson <- x$Log.UnknownIncomeDebtRatio - x$Log.HouseholdSize
  x$Log.UnknownIncomeDebtRatioPerLine <- x$Log.UnknownIncomeDebtRatio - log1p(x$NumberOfOpenCreditLinesAndLoans)
  x$Log.UnknownIncomeDebtRatioPerRealEstateLine <- x$Log.UnknownIncomeDebtRatio - log1p(x$NumberRealEstateLoansOrLines)
  x$Log.UnknownIncomeDebtRatioPerDelinquency <- x$Log.UnknownIncomeDebtRatio - log1p(x$NumberOfTimesPastDue)
  x$Log.UnknownIncomeDebtRatioPer90DaysLate <- x$Log.UnknownIncomeDebtRatio - log1p(x$NumberOfTimes90DaysLate)

"""
print 'TODO'

TODO


In [141]:
"""
  x$Log.NumberRealEstateLoansOrLines <- log(x$NumberRealEstateLoansOrLines)
  x$Log.NumberRealEstateLoansOrLines[!is.finite(x$Log.NumberRealEstateLoansOrLines)] <- 0
  x$NumberRealEstateLoansOrLines <- NULL
"""

print 'TODO'

TODO


In [142]:
"""
  x$NumberOfOpenCreditLinesAndLoans <- NULL
  
  x$NumberOfTimesPastDue <- NULL
  x$NumberOfTimes90DaysLate <- NULL
  x$NumberOfTime30.59DaysPastDueNotWorse <- NULL
  x$NumberOfTime60.89DaysPastDueNotWorse <- NULL
"""

print 'TODO'

TODO


In [143]:
"""
  x$LowAge <- as.integer(x$age < 18)
  x$Log.age <- log(x$age - 17)
  x$Log.age[x$LowAge == 1] <- 0
  x$age <- NULL
"""

print 'TODO'

TODO
